# Same differential evolution implementation with gaussian noise on mass constraint


In [ ]:
%reset
import numpy as np
import csv
from scipy.optimize import * #must use gentest python 3.12.0 and scipy 1.31.1
import sys
import matplotlib.pyplot as plt
np.set_printoptions(threshold=sys.maxsize)
import datetime
from scipy.stats import skewnorm

In [ ]:
#import cell

#tech
tech_array = []

#take data from tech csv and convert it to an array
with open('Found_Technologies.csv', 'rt') as f:
    array_x = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_x:
        tech_array.append(line)

#take data from scores tech csv and convert it to an array
tech_score = []
with open('Shortfalls_scores.csv', 'rt') as f:
    array_x = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_x:
        tech_score.append(line)

#print(tech_score)


In [ ]:
#data processing cell

#Parse shortfall data 
tech_array_mod          = tech_array[1:]                                    #cut off title score
tech_mass_array         = np.asarray([i[-1] for i in tech_array_mod],float)      #Masses of techs 0 indexed
tech_power_array        = np.asarray([i[-2] for i in tech_array_mod],float)      #power consumption of techs 0 indexed
shortfall_array_temp    = [np.array(i[1:-2]) for i in tech_array_mod]       
shortfall_array         = []
for i in shortfall_array_temp:
    arr = []
    for j in i:
        s = j
        y = ""
        for char in s:
            if char.isdigit():
                y += char
        num = int(y)
        arr.append(num)
    
    shortfall_array.append(arr)

shortfall           = np.asarray(tech_score[1:],float)
shortfall_matrix    = np.zeros((len(shortfall_array),len(shortfall)))
for i in range(len(shortfall_matrix)):
    for j in shortfall_array[i]:
        shortfall_matrix[i][j-1] = 1

shortfall_matrix = np.transpose(shortfall_matrix) #matrix of shortfalls where the column are a tech and the value (0 or 1) represents of that index of a shortfall is completed

shortfall_integrated    = shortfall[0:, 1]  #Scores from NASA 0 indexed
shortfall_consequence   = shortfall[0:, 2]  #Scores from consequence axis 0 indexed
shortfall_liklihood     = shortfall[0:, 3]  #Scores from liklihood axis 0 indexed


beta = 185 #kg/kW Power to mass ratio
max_mass    = 40000 #kg
min_mass    = 15000 #kg
mass_ratio  =  0.3 #kg

In [ ]:
#random guassian
def sample_skewed_gaussian(peak, skew, scale=2, size=1, domain=(0, 60000)):
    loc = peak
    samples = []
    max_attempts = 10000 
    x = np.linspace(*domain, 1000)
    pdf_vals = skewnorm.pdf(x, a=skew, loc=loc, scale=scale)
    pdf_max = np.max(pdf_vals)
    attempts = 0
    while len(samples) < size and attempts < max_attempts:
        x_try = np.random.uniform(*domain)
        y_try = np.random.uniform(0, pdf_max)
        pdf_val = skewnorm.pdf(x_try, a=skew, loc=loc, scale=scale)
        if y_try <= pdf_val:
            samples.append(x_try)
        attempts += 1
    if len(samples) < size:
        raise RuntimeError("Sampling failed — try increasing max_attempts or adjusting parameters.")
    return np.array(samples) if size > 1 else samples[0]

In [ ]:
M = []
s = []
m = []
for j in range(0,100):

    ML = sample_skewed_gaussian(15000,skew=0,scale=2500)
    MU = sample_skewed_gaussian(40000,skew=0,scale=2500)
    
    max_mass    = MU #kg
    min_mass    = ML #kg
    mass_ratio  =  0.3 #kg
    M.append([ML,MU])
    print(M)
    def score_tech(mission, w1=2.3,w2=7,w3=4,w4=4,check_mass=True):
   
        mass_max_tech = max_mass*(1-mass_ratio) #in kg
        mass_min_tech = min_mass*(1-mass_ratio) #in kg

        #Mass check
        if check_mass and (tech_mass_array.dot(mission) + beta*tech_power_array.dot(mission) <= mass_min_tech or tech_mass_array.dot(mission) + beta*tech_power_array.dot(mission) > mass_max_tech):
            return abs(tech_mass_array.dot(mission) + beta*tech_power_array.dot(mission))

        #normlizers
        max_number      = 93        #normalize number score 0-1 
        max_integrated  = 8.1035    #normalize integrated score 0-1
        max_consequence = 5         #normlize consquence score 0-1
        max_liklihood   = 5         #normlize consquence score 0-1  

        #normlize weights to sum to 10
        W  = w1+w2+w3+w4 
        w1 = 10*w1/W
        w2 = 10*w2/W
        w3 = 10*w3/W
        w4 = 10*w4/W

        mission                     = mission.astype(int) #Convete to ints
        completed                   = np.clip(shortfall_matrix @ mission, a_min=None, a_max=1) #A shortfall is either completed or not
        completed_number_score      = np.linalg.norm(completed)**2                             #Number of shortfalls completed
        normalized_completed        = completed/completed_number_score                         #Normlizes completed array to weight each score

        completed_integrated_score  = shortfall_integrated  @ normalized_completed             #Generates integrated score
        completed_consequence_score = shortfall_consequence @ normalized_completed             #Generates the consquences score
        completed_liklihood_score   = shortfall_liklihood   @ normalized_completed             #Generates the liklihood score

        #score calculation
        score = w1*completed_number_score/max_number + w2*completed_integrated_score/max_integrated + w3*completed_consequence_score/max_consequence + w4*completed_liklihood_score/max_liklihood
        return -score
    
    scores_per_tech = np.zeros(len(tech_array)-1)
    for i in range(len(scores_per_tech)):
        input = np.zeros(len(tech_array)-1)
        input[i] = 1
        scores_per_tech[i] = score_tech(input,check_mass=False)
    indcies_tech = np.argsort(scores_per_tech)
    scores_per_tech_sort = np.sort(scores_per_tech) 

    #rint(scores_per_tech_sort)
    #print(indcies_tech)
    mass = 0
    bring_array_tech = np.zeros(len(tech_array)-1)
    for i in range(len(indcies_tech)):
        if mass < 0.5*(max_mass+min_mass)*(1-mass_ratio):
            bring_array_tech[indcies_tech[i]] = 1
            mass = tech_mass_array.dot(bring_array_tech) + beta*tech_power_array.dot(bring_array_tech)
    print(bring_array_tech)
    print(score_tech(bring_array_tech))

    bounds_tech = Bounds(lb=np.zeros(len(shortfall_array)), ub=np.ones(len(shortfall_array)))     #we either bring or we dont
    seed = np.random.randint(9999999)
    res_tech    = differential_evolution(func=score_tech, bounds=bounds_tech, x0=bring_array_tech,integrality=np.ones(len(tech_array)-1),updating='deferred',tol=0.01,init='sobol',popsize=16,recombination=0.05,mutation=(0.7,1.7),maxiter=100000,seed=seed)
    s.append(seed)
    m.append(res_tech)

In [ ]:
pops = []
for i in m:
    pops.append(i['x'].tolist())

In [ ]:
now = datetime.datetime.now()
formatted_time = now.strftime("%d-%m-%Y_%H_%M_%S")
import json
data = {
    'seeds' : s,
    'populations' : pops,
    'mass' : M
}
with open('100_guassian_std_0.5_tech_mass' + formatted_time + '.json', 'w') as f:
    json.dump(data, f, indent=4)  

In [ ]:
for i in m:
    print(i)

In [ ]:
outputs_tech = []
count = 0
for i in m:
    if i['success']:
        outputs_tech.append(i['x'])
        count += 1

result = []
with open('Techs_to_Bring_via_sort_22-10-2024_14_56_55.csv', 'rt') as f:
    array_x = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_x:
        result.append(line)
result = result[1:]
objectivs_names_temp = []
for i in result:
    objectivs_names_temp.append(i[1])
objectivs_names_temp = objectivs_names_temp[0:-1]

highlight_indices = objectivs_names_temp

percentages = np.mean(outputs_tech, axis=0) * 100
sorted_indices = np.argsort(-percentages)
sorted_percentages = percentages[sorted_indices]
indices = [row[0] for row in tech_array_mod]
indices = [indices[i] for i in sorted_indices]
plt.figure(figsize=(15,5))
plt.bar(indices, sorted_percentages, color='grey')
plt.ylabel('Occurrence Percentage')
plt.title(f'Common Technologies over {count} trials')
plt.xticks(indices, rotation=90, fontsize=7)

ax = plt.gca()
for bar, tick_label in zip(ax.patches, ax.get_xticklabels()):
    if tick_label.get_text() in highlight_indices:
        tick_label.set_color('red')
        #bar.set_color('darkred') 
plt.show()